### CS 210: Data Management for Data Science
#### Fall 2023

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

## Homework 5: <font color="blue">SQL Database</font>

<p>Due Wednesday, December 13 at 11 PM in Codebench</font>
<p><font color="red">No Late Submissions Accepted</font></p>
</h3>
</center>
You may work on the assignment in pairs that you choose.
<hr>

### Overview

<p>Implement a Music relational (SQL) database, of the kind that might be used
by Spotify or Amazon Music. The database has artists, albums, songs, users, 
playlists, and ratings.

<p>You are allowed up to 3 submissions. <font color="red">Only the last submission will be graded.</font>
   
You may populate the tables with data for your own testing, but we 
<b>do not</b> want you to turn in any of the data, or the results
of any of your queries. We are only asking for the
document with the required SQL statements for table creation and queries.



---

<h3>Part I: Database Schema (50 pts)</h3>

<p>You are given the following description of the entities that need to be
stored in the database. Your task is to design a database schema (set of tables)
to store these entities. 

<p><font color="red">Your schema must be minimally redundant in storing 
data.</font>
In other words, you should build a set of tables that minimize the repetition
of data, by using foreign keys - credit will be in accordance with this.

<ul>
<p><li><b>Artist</b>: An individual or a group/band, uniquely identified by
their name.
An artist might release albums, as well as songs that are not in albums (singles).

<p><li><b>Song</b>: A song has a title and is performed by an artist, either as
a part of an album, or as a single that's not part of an album. Every song
in an album has the release date of the album, but a single song has its
own release date.

A song title is
unique to an artist (the same artist records a song exactly once), but the
title may be shared by multiple artists (i.e. covers).

<p>A song belongs to one or more genres. For example, a song could
be in a single genre, such as <em>R &amp; B</em>, or could be in multiple
genres such as <em>Pop</em> and <em>Rock</em>. Genres are pre-defined, and every
song must be in at least one genre.
Also, songs in an album need not all be in the same genre.

<p><li><b>Album</b>: An album is a collection of songs released by an artist, on a 
certain date. For example, the album <em>Achtung Baby</em> was made by 
the artist (band) <em>U2</em>, released on <em>November 19, 1991</em>.
An album name is not unique, but the combination of album
name and artist name is unique.

<p><li><b>User</b>: A user is uniquely identified by their <em>username</em>. 
A user can optionally have one or more playlists, and optionally have ratings
for songs, albums, or playlists.
In other words, it's possible that a user has no playlists,
and hasn't given any ratings.

<p><li><b>Playlist</b>: A user can make any number of playlists of songs. 
Note: A playlist may not include an entire album, only individual songs.
Each song is either from some album, or a single that's not in any album. 

<p>Every playlist has a title, and a date+time when it was created. A playlist
may be modified any number of times after creation by adding or removing
songs, but the title and date+time will not change.

<p>The title of a playlist is not unique since different users might create 
playlists with the same title. However, a user's playlists will have unique
titles.

<p><li><b>Rating</b>: A user could rate an album, a song (even if it's in an album), 
or a playlist. A rating is limited to 1,2,3,4, or 5 (numeric), and is made 
on a specific date. 
</ul>

<p>Your database structure should have the most appropriate data type and size
for each column in each table.

<p>For size of data, think of a realistic online music service and imagine
how many songs/artists/albums/playlists/users/ratings it might have to support.
The idea is to use the least amount of storage space for each column that
will be able to store the entire range of foreseeable values.

<p>Make sure you define and specify all primary keys, foreign keys, unique
valued columns or unique valued combination of columns, and null/non-null 
properties for columns.

<p>Make sure that when you submit this notebook, 
it contains the <tt>create table</tt> 
statement for each of the tables you
create in the database. If you don't have the full
create statement for a table, you will not get credit for it.<br>

<p><b>Note</b>: 
When you test your design in sqlite3, you might use <tt>alter table</tt> 
statements after the initial create. However, for the submission, you are
required to rewrite the whole sequence as a single <tt>create table</tt>
statement per table.



In [37]:
import sqlite3

conn = sqlite3.connect('music_database.db')
cursor = conn.cursor()

cursor.execute('''
    DROP TABLE Artist
''')
cursor.execute('''
    CREATE TABLE Artist (
        artist_id INTEGER PRIMARY KEY,
        name VARCHAR(255) UNIQUE NOT NULL
    )
''')

cursor.execute('''
    DROP TABLE Genre
''')

cursor.execute( ''' 
    CREATE TABLE Genre (
        genre_id INTEGER PRIMARY KEY,
        name VARCHAR(50) UNIQUE NOT NULL
    )
''')

cursor.execute('''
    DROP TABLE Song
''')

cursor.execute( ''' 
    CREATE TABLE Song (
        song_id INTEGER PRIMARY KEY,
        title VARCHAR(255) NOT NULL,
        artist_id INTEGER,
        release_date DATE,
        is_single BOOLEAN DEFAULT FALSE,
        FOREIGN KEY (artist_id) REFERENCES Artist(artist_id)
    );
''')

cursor.execute('''
    DROP TABLE SongGenres
''')

cursor.execute( ''' 
    CREATE TABLE SongGenres (
        song_id INTEGER,
        genre_id INTEGER,
        PRIMARY KEY (song_id, genre_id),
        FOREIGN KEY (song_id) REFERENCES Song(song_id),
        FOREIGN KEY (genre_id) REFERENCES Genre(genre_id)
    );
''')

cursor.execute('''
    DROP TABLE Album
''')

cursor.execute( ''' 
    CREATE TABLE Album (
        album_id INTEGER PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        artist_id INTEGER,
        release_date DATE NOT NULL,
        FOREIGN KEY (artist_id) REFERENCES Artist(artist_id)
    );
''')

cursor.execute('''
    DROP TABLE AlbumSongs
''')

cursor.execute( ''' 
    CREATE TABLE AlbumSongs (
        album_id INTEGER,
        song_id INTEGER,
        PRIMARY KEY (album_id, song_id),
        FOREIGN KEY (album_id) REFERENCES Album(album_id),
        FOREIGN KEY (song_id) REFERENCES Song(song_id)
    );

''')

cursor.execute('''
    DROP TABLE User
''')
cursor.execute( ''' 
    CREATE TABLE User (
        user_id INTEGER PRIMARY KEY,
        username VARCHAR(50) UNIQUE NOT NULL
    );

''')

cursor.execute('''
    DROP TABLE Playlist
''')
cursor.execute( ''' 
    CREATE TABLE Playlist (
        playlist_id INTEGER PRIMARY KEY,
        title VARCHAR(255) NOT NULL,
        user_id INTEGER,
        created_at DATETIME NOT NULL,
        FOREIGN KEY (user_id) REFERENCES User(user_id)
    );
''')

cursor.execute('''
    DROP TABLE PlaylistSongs
''')

cursor.execute( ''' 
    CREATE TABLE PlaylistSongs (
        playlist_id INTEGER,
        song_id INTEGER,
        PRIMARY KEY (playlist_id, song_id),
        FOREIGN KEY (playlist_id) REFERENCES Playlist(playlist_id),
        FOREIGN KEY (song_id) REFERENCES Song(song_id)
    );
''')

cursor.execute('''
    DROP TABLE Rating
''')
cursor.execute( ''' 
    CREATE TABLE Rating (
        rating_id INTEGER PRIMARY KEY,
        user_id INTEGER,
        target_id INTEGER,
        target_type VARCHAR(20) NOT NULL, -- 'Album', 'Song', or 'Playlist'
        rating_value INTEGER CHECK (rating_value >= 1 AND rating_value <= 5) NOT NULL,
        rating_date DATE NOT NULL,
        FOREIGN KEY (user_id) REFERENCES User(user_id),
        CHECK (target_type IN ('Album', 'Song', 'Playlist'))
    );
''')

---

### Part II: Queries (5 points each)

<p>Every query must be written in a <font color="red">single</font>
SQL statement.
So, for example, you can have nested or multiple SQLs for a query, 
provided you can write
it all up with a single terminating semicolon in a SQL client session.
No Python code other than to execute the queries and display the results. 

<p>For any of the queries:
<ul>
<li>If the result might require breaking ties, then
<em>unless otherwise specified in the query</em>, 
let the ties be broken arbitrarily (you 
need not do anything explicit).
<li>If the result has fewer than the required number of entities,
report all of them.
<li>For all queries that ask for 'top n' or 'most', the result must appear
from highest ranked to lowest ranked.

</ul>


<p>Type the SQL queries in the cell following each problem description below. 

### Write queries for the following: 

#### 1. Which 3 genres are most represented in terms of number of songs in that genre?<p>
The result must have two columns, named <tt>genre</tt> and 
<tt>number_of_songs</tt>.


In [39]:
import sqlite3

conn = sqlite3.connect('music_database.db')

# Run the SQL query
query = '''
    SELECT G.name AS genre, COUNT(SG.song_id) AS number_of_songs
    FROM Genre G
    LEFT JOIN SongGenres SG ON G.genre_id = SG.genre_id
    GROUP BY G.genre_id
    ORDER BY number_of_songs DESC
    LIMIT 3;
'''

result = conn.execute(query).fetchall()

# Display the result
result

[]

#### 2. Find names of artists who have songs that are in albums as well as outside of albums (singles).<p>
The result must have one column, named
<tt>artist_name</tt>. 

In [40]:
import sqlite3

conn = sqlite3.connect('music_database.db')

# Run the SQL query
query = '''
    SELECT DISTINCT A.name AS artist_name
    FROM Artist A
    JOIN Song S ON A.artist_id = S.artist_id
    WHERE S.is_single = FALSE
       OR S.is_single IS NULL;
'''

result = conn.execute(query).fetchall()

# Display the result
result

[]

#### 3. What were the top 10 albums (albums with highest average user rating) in the period 1990-1999? 
Break ties using alphabetical order of
album names. (Period refers to the rating date, NOT the date of release).<p>
The result must have two columns, named <tt>album_name</tt> and 
<tt>average_user_rating</tt>.

In [41]:
import sqlite3

conn = sqlite3.connect('music_database.db')

# Run the SQL query
query = '''
    SELECT A.name AS album_name, AVG(R.rating_value) AS average_user_rating
    FROM Album A
    JOIN Rating R ON A.album_id = R.target_id AND R.target_type = 'Album'
    WHERE strftime('%Y', R.rating_date) BETWEEN '1990' AND '1999'
    GROUP BY A.album_id, A.name
    ORDER BY average_user_rating DESC, A.name
    LIMIT 10;
'''

result = conn.execute(query).fetchall()

# Display the result
result


[]

#### 4. Which were the top 3 most rated genres
(this is the number of ratings
of songs in genres, not the actual rating scores) in the years 1991-1995?
(Years refers to the rating date, NOT the date of release).<p>
The result must have two columns, named <tt>genre_name</tt> and
<tt>number_of_song_ratings</tt>. 

In [42]:
import sqlite3

conn = sqlite3.connect('music_database.db')


query = '''
    SELECT G.name AS genre_name, COUNT(R.rating_id) AS number_of_song_ratings
    FROM Genre G
    JOIN SongGenres SG ON G.genre_id = SG.genre_id
    JOIN Song S ON SG.song_id = S.song_id
    LEFT JOIN Rating R ON S.song_id = R.target_id AND R.target_type = 'Song'
    WHERE strftime('%Y', R.rating_date) BETWEEN '1991' AND '1995'
    GROUP BY G.genre_id
    ORDER BY number_of_song_ratings DESC
    LIMIT 3;
'''

result = conn.execute(query).fetchall()

# Display the result
result


[]

#### 5. Which users have a playlist that has an average rating of 4.0 or more?
(This is the average of the ratings for the playlist.)
A user may appear multiple times in the result if more than one of their
playlists make the cut.<p>
The result must 3 columns
named <tt>username</tt>, <tt>playlist_title</tt>, <tt>average_playlist_rating</tt>


In [43]:
import sqlite3

conn = sqlite3.connect('music_database.db')

# Run the SQL query
query = '''
    SELECT U.username, P.title AS playlist_title, AVG(R.rating_value) AS average_playlist_rating
    FROM User U
    JOIN Playlist P ON U.user_id = P.user_id
    LEFT JOIN PlaylistSongs PS ON P.playlist_id = PS.playlist_id
    LEFT JOIN Song S ON PS.song_id = S.song_id
    LEFT JOIN Rating R ON S.song_id = R.target_id AND R.target_type = 'Song'
    GROUP BY U.user_id, P.playlist_id
    HAVING AVG(R.rating_value) >= 4.0;
'''

result = conn.execute(query).fetchall()

# Display the result
result


[]

#### 6. Who are the top 5 users that have rated the most songs? 
This includes all song ratings only, not album or playlist ratings. 
The result must have 2 columns, named <tt>username</tt> and
<tt>number_of_ratings</tt>. 

In [44]:
import sqlite3

conn = sqlite3.connect('music_database.db')

# Run the SQL query
query = '''
    SELECT U.username, COUNT(R.rating_id) AS number_of_ratings
    FROM User U
    LEFT JOIN Rating R ON U.user_id = R.user_id AND R.target_type = 'Song'
    GROUP BY U.user_id
    ORDER BY number_of_ratings DESC
    LIMIT 5;
'''

result = conn.execute(query).fetchall()

# Display the result
result


[]

#### 7. Find the top 10 most prolific artists (most songs) in the years 1990-2010. 
Count each song in an album individually. <p>
The result must have 2 columns, named <tt>artist_name</tt> and
<tt>number_of_songs</tt>. 

#### 7. Find the top 10 most prolific artists (most songs) in the years 1990-2010. 
Count each song in an album individually. <p>
The result must have 2 columns, named <tt>artist_name</tt> and
<tt>number_of_songs</tt>. 

In [45]:
import sqlite3

# Connect to your SQLite database (replace 'music_database.db' with your actual database file)
conn = sqlite3.connect('music_database.db')

# Run the SQL query
query = '''
    SELECT A.name AS artist_name, COUNT(S.song_id) AS number_of_songs
    FROM Artist A
    JOIN Album AL ON A.artist_id = AL.artist_id
    JOIN AlbumSongs ALS ON AL.album_id = ALS.album_id
    JOIN Song S ON ALS.song_id = S.song_id
    WHERE strftime('%Y', AL.release_date) BETWEEN '1990' AND '2010'
    GROUP BY A.artist_id
    ORDER BY number_of_songs DESC
    LIMIT 10;
'''

result = conn.execute(query).fetchall()

# Display the result
result


[]

#### 8. Find the top 10 songs that are in the most playlists. 
Break ties in alphabetical order of song titles.<p>
The result must have a 2 columns, named <tt>song_title</tt>
and <tt>number_of_playlists</tt>. 

In [46]:
import sqlite3

# Connect to your SQLite database (replace 'music_database.db' with your actual database file)
conn = sqlite3.connect('music_database.db')

# Run the SQL query
query = '''
    SELECT S.title AS song_title, COUNT(PS.playlist_id) AS number_of_playlists
    FROM Song S
    LEFT JOIN PlaylistSongs PS ON S.song_id = PS.song_id
    GROUP BY S.song_id
    ORDER BY number_of_playlists DESC, S.title
    LIMIT 10;
'''

result = conn.execute(query).fetchall()

# Display the result
result


[]

#### 9. Find the top 20 most rated singles. 
(songs that are not part of an album).
Most rated refers to number of ratings, not actual rating scores.
The result must have 3 columns, named <tt>song_title</tt>,
<tt>artist_name</tt>, <tt>number_of_ratings</tt>. 

In [47]:
import sqlite3

# Connect to your SQLite database (replace 'music_database.db' with your actual database file)
conn = sqlite3.connect('music_database.db')

# Run the SQL query
query = '''
    SELECT S.title AS song_title, A.name AS artist_name, COUNT(R.rating_id) AS number_of_ratings
    FROM Song S
    JOIN Artist A ON S.artist_id = A.artist_id
    LEFT JOIN Rating R ON S.song_id = R.target_id AND R.target_type = 'Song'
    WHERE S.is_single = TRUE
    GROUP BY S.song_id
    ORDER BY number_of_ratings DESC
    LIMIT 20;
'''

result = conn.execute(query).fetchall()

# Display the result
result


[]

#### 10. Find all artists who stopped making music after 1993.<p>
Include all artists that have no songs or albums with a release date 
after 1993. 
The result should be a single column named <tt>artist_title</tt>. 

In [48]:
import sqlite3

# Connect to your SQLite database (replace 'music_database.db' with your actual database file)
conn = sqlite3.connect('music_database.db')

# Run the SQL query
query = '''
    SELECT DISTINCT A.name AS artist_name
    FROM Artist A
    LEFT JOIN Album AL ON A.artist_id = AL.artist_id
    LEFT JOIN Song S ON A.artist_id = S.artist_id
    WHERE COALESCE(AL.release_date, S.release_date, '1993-01-01') <= '1993-12-31';
'''

result = conn.execute(query).fetchall()

# Display the result
result


[]